# RN-A · Robustez al radio de búsqueda (subsets por separación)
Calcula N/P50/P68/P95 para cortes de separación `r ∈ {0.4, 0.8, 1.2, 2.0}"` sobre el parquet ya deduplicado.
Añade el bloque `radii_check` al JSON y muestra una tabla.
**Nota:** Verifica estabilidad; no estima FDR por *offset grid* (eso requiere catálogos originales).

In [3]:
from pathlib import Path
import json
import numpy as np, pandas as pd
from IPython.display import display

ROOT = Path.home()/ 'portafolio-rubin'
PARQUET = ROOT/'data/47tuc_dp1/rnA_matched_minimal.parquet'
JSONP   = ROOT/'data/47tuc_dp1/rnA_metrics.json'
JSON_REL= ROOT/'release/rnA_v1.0/rnA_metrics.json'
df = pd.read_parquet(PARQUET)
sep = df['separation_arcsec'].to_numpy()
radii = [0.4, 0.8, 1.0, 1.2, 1.5, 2.0]
rows = []
for r in radii:
    m = sep <= r
    s = sep[m]
    if len(s)==0:
        rows.append((r, 0, np.nan, np.nan, np.nan))
    else:
        rows.append((r, int(m.sum()), float(np.percentile(s,50)), float(np.percentile(s,68)), float(np.percentile(s,95))))
tab = pd.DataFrame(rows, columns=['radius_arcsec','N','P50','P68','P95'])
display(tab)

,radius_arcsec,N,P50,P68,P95
0,0.4,1090,0.050877,0.053147,0.084029
1,0.8,1096,0.050970,0.053236,0.097704
2,1.0,1100,0.050970,0.053236,0.099697
3,1.2,1102,0.050970,0.053236,0.102142
4,1.5,1104,0.050970,0.053236,0.108183
5,2.0,1113,0.050970,0.053325,0.115842


In [4]:
def merge_radii(path, tab):
    try:
        data=json.loads(path.read_text())
    except Exception:
        data={}
    data['radii_check']={
        'radius_arcsec': tab['radius_arcsec'].tolist(),
        'N': tab['N'].tolist(),
        'P50_arcsec': tab['P50'].tolist(),
        'P68_arcsec': tab['P68'].tolist(),
        'P95_arcsec': tab['P95'].tolist(),
    }
    path.write_text(json.dumps(data, indent=2))
    return data

JSONP = Path(JSONP); JSON_REL = Path(JSON_REL)
updated = merge_radii(JSONP, tab)
if JSON_REL.exists():
    merge_radii(JSON_REL, tab)
display(updated)

{'N_pairs': 1113,
 'P50_arcsec': 0.050969643129182965,
 'P68_arcsec': 0.05332470894897676,
 'P95_arcsec': 0.11584190795115097,
 'P50_ci95_arcsec': [0.050690864503642435, 0.051430916269940744],
 'P95_ci95_arcsec': [0.09780514929323501, 0.17069029791484716],
 'bootstrap_B': 5000,
 'bootstrap_seed': 47,
 'method_note': '1:1 dentro de 2″; percentiles sobre separaciones; IC-95% por bootstrap.',
 'units': 'arcsec',
 'generator': {'script': 'notebooks/47tuc/rnA_per_coordinate_metrics.ipynb',
  'version': 'v1.1-rnA'},
 'P68_ci95_arcsec': [0.05296920692603931, 0.053765781052101044],
 'per_coordinate': {'rms_ra_mas': 130.90993556974593,
  'rms_dec_mas': 146.93924562235915,
  'rms_ra_ci95_mas': [97.65973016695716, 162.34075753107206],
  'rms_dec_ci95_mas': [104.34233398890363, 185.35156993450624],
  'note': 'Δα⋆ incluye cosδ; unidades mas por coordenada.'},
 'radii_check': {'radius_arcsec': [0.4, 0.8, 1.0, 1.2, 1.5, 2.0],
  'N': [1090, 1096, 1100, 1102, 1104, 1113],
  'P50_arcsec': [0.05087688664